In [79]:
import numpy as np
from collections import deque, defaultdict 
from tqdm import tqdm_notebook as tqdm

In [157]:
frame = 1000
C = deque()
H_LRU = deque(maxlen=frame)
H_LFU = deque(maxlen=frame)
W_LFU = 1
W_LRU = 1
recency = deque()
frequency = defaultdict(int)
History = deque(maxlen=frame)
d = 0.005**(1/frame)
lam = 0.45
weight = {}
W_LFU = 1
W_LRU = 1
hit,miss = 0,0

In [158]:
blocktrace = np.random.randint(5000, size=(10000))

In [159]:
LRU = lambda cache: cache[0]

In [160]:
def LFU(cache):
    global frequency
    tempFrequency = {i:j for i,j in frequency.items() if i in cache}
    return min(tempFrequency, key=tempFrequency.get)

In [164]:
def Lecar(block):
    global frame, C, H_LRU, H_LFU, weight, History, hit, miss
    
    if block in C:
        C.remove(block)
        C.append(block)
        hit += 1
    else:
        miss += 1
        updated_wait(block)
        if block in H_LRU:
            H_LRU.remove(block)
        elif block in H_LFU:
            H_LFU.remove(block)
        if len(C) == frame:
            action = max(weight, key=weight.get)
            if action == 'LRU':
                #print('LRU')
                if len(H_LRU) == frame:
                    H_LRU.remove(LRU(H_LRU))
                H_LRU.append(LRU(C))
                C.remove(LRU(C))
                History.append(LRU(C))
            else:
                #print('LFU')
                if len(H_LFU) == frame:
                    H_LFU.remove(LRU(H_LFU))
                H_LFU.append(LFU(C))
                C.remove(LFU(C))
                History.append(LFU(C))

        C.append(block)

In [162]:
def updated_wait(q):
    global d, lam, W_LRU, W_LFU
    if q in History:
        t = len(History) - History.index(q)
    else:
        t = 0
    r = d**t
    if q in H_LRU:
        W_LFU = W_LFU * np.exp(lam*r)
    elif q in H_LFU:
        W_LRU = W_LRU * np.exp(lam*r)
    weight['LRU']  = W_LRU / (W_LRU + W_LFU)
    weight['LFU'] = 1 - weight['LRU']

In [163]:
for i in tqdm(blocktrace):
    frequency[i] += 1
    if len(C) == frame:
        Lecar(i)
    else:
        C.append(i)
print(hit/(hit+miss))


0.1968888888888889


In [52]:
blocktrace

array([ 6, 17, 24, 18, 20,  8, 20, 17,  9,  1, 21, 21,  1, 24, 11,  5, 23,
       16,  2,  0, 22, 19, 23,  3,  1,  6,  0,  4,  0, 23,  4, 10,  7,  2,
        4,  8,  2, 13, 15, 11, 13, 16,  7, 20,  3,  0, 16,  2,  3,  7, 16,
        3,  5, 17, 14,  9, 14, 18, 20, 20, 22,  2,  3,  6,  2, 24, 13,  1,
       20, 13, 19,  0, 17, 12,  5,  2, 24, 18, 17, 18,  6,  2, 15,  7, 13,
       17, 24,  7, 10, 17,  7, 17, 22, 23, 15, 24,  6, 21, 17,  8])

In [29]:
max(a,b)

4

In [119]:
def LRU2(blocktrace, frame):
    cache = set()
    recent_block = np.array([], dtype=int)
    
    hit, miss = 0, 0
    
    #print("block", "\t\t hit", "\t\t miss", "\t\t cache")
    for block in tqdm(blocktrace, desc="LRU", leave=False):
    
        if block in cache:
            recent_block = np.delete(recent_block, np.argwhere(recent_block==block))
            recent_block = np.append(recent_block, block)
            hit += 1
            
        
        elif len(cache) < frame:
            recent_block = np.append(recent_block, block)
            cache.add(block)
            #miss += 1

        else:
            cache.remove(recent_block[0])
            recent_block = recent_block[1:]
            recent_block = np.append(recent_block, block)
            cache.add(block)
            miss += 1
        #print(block, "\t\t", hit, "\t\t", miss, "\t\t", cache)

    hitrate = hit / (hit + miss)
    #print("\n\nhitrate: ", hitrate)
    return hitrate

In [156]:
LRU2(blocktrace, 1000)

0.21288888888888888

In [ ]:
0.2013006503251626